In [1]:
import time
import datetime
from mkndaq.inst.neph import NEPH
import schedule
import threading

cfg = {'ne300': {
                'type': 'NE300',
                'serial_number': '23-0690',
                'serial_id': 0,
                'protocol': 'acoem',
                'socket': {
                    'host': '192.168.0.50',
                    'port': 32783,
                    'timeout': 10,
                },
                'get_data_interval': 3, # this is the logger retrieval interval
                'zero_span_check_interval': 13,
                'zero_check_duration': 5,
                'span_check_duration': 3,
                'staging_zip': True,  
                'verbosity': 2,  # 0: silent, 1: medium, 2: full          
            },
            'reporting_interval': 10,
            'logs': 'mkndaq/logs',
            'data': 'mkndaq/data',
            'staging': {
                'path': 'mkndaq/staging',
            },
}
ne300 = NEPH('ne300', cfg, verbosity=0)
# # Initialize NEPH (name: ne300  S/N: 23-0690)
#   Instrument identified itself as '{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}'.

# Initialize NEPH (name: ne300  S/N: 23-0690)
  - Instrument identified itself as '{'id': 'ACOEM Aurora NE-300 Sub-Type: 0 Range: 0 Build: 158 Branch: 300'}'.
  - Instrument current operation: ambient.
2024-05-28 14:52:07 [ne300] dtm found: 2024-05-28 14:52:23 > dtm after set: 2024-05-28 14:52:23.


In [2]:
def run_threaded(job_func):
    """Set up threading and start job.

    Args:
        job_func ([type]): [description]
    """
    job_thread = threading.Thread(target=job_func)
    job_thread.start()

# align start with a 10' timestamp
while int(time.time()) % 10 > 0:
    time.sleep(0.1)

fetch = 20
schedule.every(fetch).seconds.do(ne300.print_ssp_bssp)
schedule.every(cfg['ne300']['get_data_interval']).minutes.at(':00').do(run_threaded, ne300.get_new_data)
schedule.every(cfg['ne300']['zero_span_check_interval']).minutes.at(':00').do(run_threaded, ne300.do_zero_span_check)

while True:
    schedule.run_pending()
    time.sleep(1)

2024-05-28 14:53:00 [ne300] ssp|bssp (Mm-1) r: -0.1895|-0.6481 g: 0.8578|0.1052 b: 1.1124|0.2253
2024-05-28 14:53:20 [ne300] ssp|bssp (Mm-1) r: -0.1863|-0.6404 g: 0.8514|0.1117 b: 1.1172|0.2444
2024-05-28 14:53:40 [ne300] ssp|bssp (Mm-1) r: -0.1966|-0.6349 g: 0.8525|0.1202 b: 1.0937|0.2490
2024-05-28 14:54:00 [ne300] ssp|bssp (Mm-1) r: -0.1979|-0.6318 g: 0.8489|0.1196 b: 1.1358|0.2344
2024-05-28 14:54:20 [ne300] ssp|bssp (Mm-1) r: -0.1836|-0.6201 g: 0.8386|0.1180 b: 1.1936|0.2434
2024-05-28 14:54:40 [ne300] ssp|bssp (Mm-1) r: -0.1800|-0.6149 g: 0.8351|0.1207 b: 1.1932|0.2360
2024-05-28 14:55:00 .get_new_data (name=ne300, save=True)
2024-05-28 14:55:00 [ne300] ssp|bssp (Mm-1) r: -0.1599|-0.6219 g: 0.8369|0.1155 b: 1.1907|0.2164
2024-05-28 14:55:20 [ne300] ssp|bssp (Mm-1) r: -0.1666|-0.6233 g: 0.8470|0.1150 b: 1.1648|0.1953
2024-05-28 14:55:40 [ne300] ssp|bssp (Mm-1) r: -0.1542|-0.6193 g: 0.8904|0.1122 b: 1.2742|0.1894
2024-05-28 14:56:00 [ne300] ssp|bssp (Mm-1) r: -0.1237|-0.6194 g: 0.9

KeyboardInterrupt: 

In [ ]:
res = ne300.get_id()
print(res)

In [ ]:
# mimick VI099 response
ne300.get_current_data(strict=True, verbosity=0)

In [4]:
ne300.get_current_data(verbosity=0)

{1: datetime.datetime(2024, 5, 28, 15, 53, 58),
 1635000: -1.5364103317260742,
 1525000: 0.68133544921875,
 1450000: 1.4025001525878906,
 1635090: -0.9471967220306396,
 1525090: 0.3199343681335449,
 1450090: -0.46452999114990234,
 5001: 27.680017471313477,
 5004: 29.230005264282227,
 5003: 12.430000305175781,
 5002: 967.9000244140625,
 4036: 3,
 4035: 1}

In [ ]:
ne300.get_values([1635000, 1525000], verbosity=2)

In [3]:
# retrieve VI099 command parameters
ne300.get_values(parameters=[1, 1635000,1525000,1450000,1635090,1525090,1450090,5001,5004,5003,5002,4036,4035])

{1: datetime.datetime(2024, 5, 28, 15, 53, 49),
 1635000: 0.01177072525024414,
 1525000: 2.3821067810058594,
 1450000: -0.07886695861816406,
 1635090: -0.7666099071502686,
 1525090: 1.0025453567504883,
 1450090: -0.25074005126953125,
 5001: 27.670007705688477,
 5004: 29.219995498657227,
 5003: 12.420000076293945,
 5002: 967.9299926757812,
 4036: 3,
 4035: 1}

In [ ]:
ne300.get_current_data(add_params=[1001, 1002, 1003, 1004, 1005, 2001, 2002])

In [ ]:
# not implemented
ne300.get_new_data()

In [ ]:
# not implemented
ne300.get_all_data()

In [ ]:
ne300.get_instr_type()

In [ ]:
ne300.get_datetime()

In [5]:
ne300.get_logging_config()

[41,
 2635000,
 2525000,
 2450000,
 2635090,
 2525090,
 2450090,
 5001,
 5002,
 5003,
 5004,
 5005,
 5006,
 5010,
 26635000,
 26525000,
 26450000,
 13525000,
 15635000,
 15525000,
 15450000,
 11635000,
 11525000,
 11450000,
 11635090,
 11525090,
 11450090,
 6007,
 6008,
 6001,
 6002,
 6003,
 6635000,
 6525000,
 6450000,
 6635090,
 6525090,
 6450090,
 5011,
 5012,
 5014,
 4035]

In [ ]:
# cycle through operating states
# 0: ambient, 1: zero, 2: span
for i in [1, 2, 0]:
    print(f"current: {ne300.get_current_operation()} > set: {i} ...")
    print(f"done: {ne300.set_current_operation(state=i)} okay")
    time.sleep(3)

In [ ]:
from datetime import datetime, timezone, timedelta
end = datetime.now(timezone.utc)
start = end - timedelta(minutes=10)
print(f"start: {start}\nend  : {end}")
data = ne300.get_logged_data(start=start, end=end, verbosity=1)
# for i in range(len(data)):
    # print(data[i])

In [ ]:
# repeat packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,1]))
response = ne300.tcpip_comm(message=message)
print(response)
display(ne300._acoem_decode_logged_data(response=response))

In [ ]:
# next packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,0]))
response = ne300.tcpip_comm(message=message)
response